In [1]:
import numpy as np
from libeq import SolverData, species_concentration

# Create SolverData object
solver_data = SolverData.load_from_pyes(
    "/Users/lorenzo/Coding/libeq/notebooks/cu_gly_solid.json"
)

In [2]:
stoichiometry = solver_data.stoichiometry
nc = solver_data.nc
concentration = species_concentration(
    np.array([[1e-3, 1e-3, 1e-3]]),
    log_beta=solver_data.log_beta,
    stoichiometry=stoichiometry,
    full=True,
)

In [3]:
# libeq
J = np.zeros((concentration.shape[0], nc, nc))
diagonals = np.einsum(
    "ij,jk->ijk", concentration[:, nc:], np.eye(concentration.shape[1] - nc)
)
# Compute Jacobian for soluble components only
J[:, :nc, :nc] = stoichiometry @ diagonals @ stoichiometry.T
J[:, range(nc), range(nc)] += concentration[:, :nc]

J

array([[[ 1.69844783e+06,  3.39669148e+06, -7.64120021e-09],
        [ 3.39669148e+06,  6.79887589e+06,  6.92737346e+03],
        [-7.64120021e-09,  6.92737346e+03,  9.38791200e+03]]])

In [6]:
# apes

d = np.zeros((concentration.shape[0], nc, nc))
_, j, k = np.indices(d.shape)
d[j == k] = (concentration[..., :nc]).flat
A = d + stoichiometry @ diagonals @ stoichiometry.T
B = stoichiometry[np.newaxis, ...] * concentration[..., nc:, np.newaxis]

KeyboardInterrupt: 